In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
def css_styling():
    styles = """
<style>
.output_png { text-align:  center; }
</style>
    """
    return HTML(styles)
css_styling()


import numpy as np
import math

### Wykład 11
# Wielowarstwowe sieci neuronowe<br/>- propagacja wsteczna błędu
04.01.2017 r.

## Sieci wielowarstwowe

a.k.a. _Artificial Neural Network_ (ANN), _Multi-Layer Perceptron_ (MLP).

<img src="nn1.png" />

## Architektura sieci

* Budowa warstwowa, najczęściej sieci jednokierunkowe i gęste.
* Liczbę i rozmiar warstw dobiera się do każdego problemu.
* Rozmiary sieci określane poprzez liczbę neuronów lub parametrów.

## Feedforward #1

* Mając daną $n$-warstwową sieć neuronową oraz jej parametry $\Theta^{(1)}, \ldots, \Theta^{(L)} $ oraz $\beta^{(1)}, \ldots, \beta^{(L)} $ liczymy:<br/><br/> 
$$a^{(l)} = g^{(l)}\left( a^{(l-1)} \Theta^{(l)} + \beta^{(l)} \right). $$

<img src="nn2.png" />

## Feedforward #2

* Funkcje $g^{(l)}$ to **funkcje aktywacji**.<br/>
Dla $i = 0$ przyjmujemy $a^{(0)} = \mathrm{x}$ (wektor wierszowy cech) oraz $g^{(0)}(x) = x$ (identyczność).

* Parametry $\Theta$ to wagi na połączeniach miedzy neuronami dwóch warstw.<br/>
Rozmiar macierzy $\Theta^{(l)}$, czyli macierzy wag na połączeniach warstw $a^{(l-1)}$ i $a^{(l)}$, to $\dim(a^{(l-1)}) \times \dim(a^{(l)})$.

* Parametry $\beta$ zastępują tutaj dodawanie kolumny z jedynkami do macierzy cech.<br/>Macierz $\beta^{(l)}$ ma rozmiar równy liczbie neuronów w odpowiedniej warstwie, czyli $1 \times \dim(a^{(l)})$.

## Feedforward #3

* **Klasyfikacja**: dla ostatniej warstwy $L$ (o rozmiarze równym liczbie klas) przyjmuje się $g^{(L)}(x) = \mathop{\mathrm{softmax}}(x)$.
* **Regresja**: pojedynczy neuron wyjściowy; funkcją aktywacji może wtedy być np. funkcja identycznościowa.

* Pozostałe funkcje aktywacji najcześciej mają postać sigmoidy, np. sigmoidalna, tangens hiperboliczny.<br/> Ale niekoniecznie, np. ReLU, leaky ReLU, maxout.

Co z uczeniem sieci?

Do trenowania wcześniejszych modeli potrzebowaliśmy:

* funkcję kosztu,
* gradienty funkcji kosztu,
* algorytm SGD.

## Propagacja wsteczna<br >- wprowadzenie

**Problem:** mając daną funkcję $f(x)$, gdzie $x$ to wektor wejściowy, chcemy obliczyć $\nabla f(x)$.

* W przypadku sieci neuronowych, $f$ to funkcja kosztu $J$, a $x$ to dane trenujące i parametry sieci.
* W praktyce interesują nas gradienty dla parametrów $\theta$ i $\beta$.
* Dane trenujące traktujemy jako dane i ustalone.

### 1. Pochodna

$$f(x,y) = x y \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = y \hspace{0.5in} \frac{\partial f}{\partial y} = x$$

**Pochodna:** miara szybkości zmian wartości funkcji względem zmian jej argumentów.

$$\frac{df(x)}{dx} = \lim_{h\ \to 0} \frac{f(x + h) - f(x)}{h}$$

Na przykład:
$x = 4, y = -3$, to $f(x,y) = -12$ i $\frac{\partial f}{\partial x} = -3$

* Jeżeli zwiększymy wartość $x$ o niewielką wartość, całe wyrażenie zmniejszy się trzykrotnie.

Analogicznie $\frac{\partial f}{\partial y} = 4$.

**Pochodna cząstkowa względem każdej zmiennej mówi jak wrażliwe jest całe wyrażenie na zmiany wartości tej zmiennej.**

Gradient $\nabla f$ to wektor pochodnych cząstkowych: 

$$\nabla f = [\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}] = [y, x]$$

Suma:
$$f(x,y) = x + y \hspace{0.3in} \rightarrow \hspace{0.3in} \frac{\partial f}{\partial x} = 1, \hspace{0.3in} \frac{\partial f}{\partial y} = 1$$

Maksimum:
$$f(x,y) = \max(x, y) \hspace{0.3in} \rightarrow \hspace{0.3in} \frac{\partial f}{\partial x} = \mathbb{1}(x >= y), \hspace{0.3in} \frac{\partial f}{\partial y} = \mathbb{1}(y >= x)$$

### 2. Funkcje złożone

$f(x,y,z) = (x + y) z$

* Wyrażenie można rozbić: $q = x + y$ i $f = q z$:
$$\frac{\partial f}{\partial q} = z, \frac{\partial f}{\partial z} = q$$ 
$$\frac{\partial q}{\partial x} = 1, \frac{\partial q}{\partial y} = 1$$

* Reguła łańcuchowa:
$$\frac{\partial f}{\partial x} = \frac{\partial f}{\partial q} \frac{\partial q}{\partial x}$$
$$\frac{\partial f}{\partial y} = \frac{\partial f}{\partial q} \frac{\partial q}{\partial y}$$

In [4]:
# wejście jest ustalone
x = -2; y = 5; z = -4

In [5]:
# krok w przód
q = x + y
f = q * z
print(q, f)

3 -12


In [6]:
# backprop dla f = q * z
dz = q
dq = z
# backprop dla q = x + y
dx = 1 * dq # reguła łańcuchowa!
dy = 1 * dq # reguła łańcuchowa!

print([dx, dy, dz])

[-4, -4, 3]


<img src="exp1.png" />

### 3. Modułowość obliczeń

Funkcja sigmoidalna:

$$f(\theta,x) = \frac{1}{1+e^{-(\theta_0 x_0 + \theta_1 x_1 + \theta_2)}}$$

$$
\begin{array}{lcl}
f(x) = \frac{1}{x} \hspace{.3in} & \rightarrow & \hspace{.4in} \frac{df}{dx} = -1/x^2 
\\[2mm]
f_c(x) = c + x \hspace{.3in} & \rightarrow & \hspace{.4in} \frac{df}{dx} = 1 
\\[2mm]
f(x) = e^x \hspace{.3in} & \rightarrow & \hspace{.4in} \frac{df}{dx} = e^x
\\[2mm]
f_a(x) = ax \hspace{.3in} & \rightarrow & \hspace{.4in} \frac{df}{dx} = a
\\[2mm]
\end{array}
$$

<img src="exp2.png" />


$$\sigma(x) = \frac{1}{1+e^{-x}} \hspace{0.3in} \rightarrow \hspace{0.3in} \frac{d\sigma(x)}{dx} = \left( 1 - \sigma(x) \right) \sigma(x)$$

In [7]:
# losowe wagi i dane
w = [2,-3,-3]
x = [-1, -2]

# krok w przód
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid

# krok w tył
ddot = (1 - f) * f # pochodna sigmoid
dx = [w[0] * ddot, w[1] * ddot]
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot]

print(dx)
print(dw)

[0.3932238664829637, -0.5898357997244456]
[-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]


### 4. Przykład

$$f(x,y) = \frac{x + \sigma(y)}{\sigma(x) + (x+y)^2}$$

In [8]:
x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!                                 #(8)

In [9]:
# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !!                  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy                                 #(1)
# done!

### 5. Wzorce zmian

* **add**: kopiuje gradient z wyjścia do każdego wejścia
* **max**: kopiuje wyjście do wejścia o największej wartości, reszta 0
* **mul**: wejściowe gradienty są mnożone przez wyjściowy i zamieniane miejscami

<img src="exp3.png" />

### Podsumowanie

* Gradient $f$ dla $x$ mówi jak zmieni się całe wyrażenie przy zmianie wartości $x$.
* Gradienty łączymy korzystając z **reguły łańcuchowej**.
* W kroku wstecz gradienty informują które części grafu powinny być zwiększone lub zmniejszone (i z jaką siłą), aby zwiększyć wartość na wyjściu.
* W kontekście implementacji chcemy dzielić funkcję $f$ na części, dla których można łatwo obliczyć gradienty.

## Uczenie wielowarstwowych sieci neuronowych

Mając algorytm SGD oraz gradienty wszystkich wag, moglibyśmy trenować każdą sieć.

* Niech:
$$\Theta = (\Theta^{(1)},\Theta^{(2)},\Theta^{(3)},\beta^{(1)},\beta^{(2)},\beta^{(3)})$$

* Funkcja sieci neuronowej z grafiki:

$$\small h_\Theta(x) = \tanh(\tanh(\tanh(x\Theta^{(1)}+\beta^{(1)})\Theta^{(2)} + \beta^{(2)})\Theta^{(3)} + \beta^{(3)})$$
* Funkcja kosztu dla regresji:
$$J(\Theta) = \dfrac{1}{2m} \sum_{i=1}^{m} (h_\Theta(x^{(i)})- y^{(i)})^2 $$

* Jak obliczymy gradienty?

$$\nabla_{\Theta^{(l)}} J(\Theta) = ? \quad \nabla_{\beta^{(l)}} J(\Theta) = ?$$

## W kierunku propagacji wstecznej

* Pewna (niewielka) zmiana wagi $\Delta z^l_j$ dla $j$-ego neuronu w warstwie $l$ pociąga za sobą (niewielką) zmianę kosztu: 

$$\frac{\partial J(\Theta)}{\partial z^{l}_j}  \Delta z^{l}_j$$

* Jeżeli $\frac{\partial J(\Theta)}{\partial z^{l}_j}$ jest duża, $\Delta z^l_j$ ze znakiem przeciwnym zredukuje koszt.
* Jeżeli $\frac{\partial J(\Theta)}{\partial z^l_j}$ jest bliska zeru, koszt nie będzie wiele poprawiony.

* Definiujemy błąd $\delta^l_j$ neuronu $j$ w warstwie $l$: 

$$\delta^l_j \equiv \dfrac{\partial J(\Theta)}{\partial z^l_j}$$ 
$$\delta^l \equiv \nabla_{z^l} J(\Theta) \textrm{ (zapis wektorowy)} $$

## Podstawowe równania propagacji wstecznej

$$
\begin{array}{ccll}
\delta^L & = & \nabla_{a^L}J(\Theta) \odot {(g^{L})}^{\prime}(z^L) & (BP1) \\[2mm]
\delta^{l} & = & ((\Theta^{l+1})^T \delta^{l+1}) \odot {{(g^{l})}^{\prime}}(z^{l}) & (BP2)\\[2mm]
\nabla_{\beta^l} J(\Theta) & = & \delta^l & (BP3)\\[2mm]
\nabla_{\Theta^l} J(\Theta) & = & a^{l-1} \odot \delta^l & (BP4)\\
\end{array}
$$


## Algorytm propagacji wstecznej

Dla jednego przykładu (x,y):

1. **Wejście**: Ustaw aktywacje w warstwie cech $a^{(0)}=x$ 
2. **Feedforward:** dla $l=1,\dots,L$ oblicz 
$$z^{(l)} = a^{(l-1)} \Theta^{(l)} + \beta^{(l)} \textrm{ oraz } a^{(l)}=g^{(l)}(z^{(l)})$$
3. **Błąd wyjścia $\delta^{(L)}$:** oblicz wektor $$\delta^{(L)}= \nabla_{a^{(L)}}J(\Theta) \odot {g^{\prime}}^{(L)}(z^{(L)})$$
4. **Propagacja wsteczna błędu:** dla $l = L-1,L-2,\dots,1$ oblicz $$\delta^{(l)} = \delta^{(l+1)}(\Theta^{(l+1)})^T \odot {g^{\prime}}^{(l)}(z^{(l)})$$
5. **Gradienty:** 
    * $\dfrac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = a_i^{(l-1)}\delta_j^{(l)} \textrm{ oraz } \dfrac{\partial}{\partial \beta_{j}^{(l)}} J(\Theta) = \delta_j^{(l)}$

W naszym przykładzie:

$$\small J(\Theta) = \frac{1}{2}(a^{(L)} - y)^2 $$
$$\small  \dfrac{\partial}{\partial a^{(L)}} J(\Theta) = a^{(L)} - y$$

$$\small \tanh^{\prime}(x) = 1 - \tanh^2(x)$$

<img src="nn3.png" />

## SGD z propagacją wsteczną

Jedna iteracja:
* Dla parametrów $\Theta = (\Theta^{(1)},\ldots,\Theta^{(L)})$ utwórz pomocnicze macierze zerowe $\Delta = (\Delta^{(1)},\ldots,\Delta^{(L)})$ o takich samych wymiarach (dla uproszczenia opuszczono wagi $\beta$).
* Dla $m$ przykładów we wsadzie (batch), $i = 1,\ldots,m$:
    * Wykonaj algortym propagacji wstecznej dla przykładu $(x^{(i)}, y^{(i)})$ i przechowaj gradienty $\nabla_{\Theta}J^{(i)}(\Theta)$ dla tego przykładu;
    * $\Delta := \Delta + \dfrac{1}{m}\nabla_{\Theta}J^{(i)}(\Theta)$
* Wykonaj aktualizację wag: $\Theta := \Theta - \alpha \Delta$

### Podsumowanie

* Algorytm pierwszy raz wprowadzony w latach 70.
* W 1986 David Rumelhart, Geoffrey Hinton, i Ronald Williams pokazali, że jest znacznie szybszy od wcześniejszych metod.
* Obecnie najpopularniejszy algorytm uczenia sieci neuronowych.

## Reverse-mode automatic differentiation

Metoda obliczania gradientu dowolnej funkcji.

Propagacja wsteczna jest szczególnym przypadkiem tej metody.

Przykład dla:

$$f(x_1,x_2) = \sin(x_1) + x_1x_2$$

<img  style="margin: auto" height="80%" src="ad0.png" />

<img  style="margin: auto" height="80%" src="ad1.png" />

## Idea za _reverse-mode auto-diff_:

* Powtarzaj zamianę pochodnych funkcji zewnętrznych korzystając z reguły łańcuchowej;
* Każde podwyrażenie jest podyktowane strukturą grafu.

$$ \small
\dfrac{\partial f}{\partial x} 
= \dfrac{\partial f}{\partial w_1} \dfrac{\partial w_1}{\partial x} 
= \left(\dfrac{\partial f}{\partial w_2} \dfrac{\partial w_2}{\partial w_1} \right) \dfrac{\partial w_1}{\partial x} 
= \left(\left(\dfrac{\partial f}{\partial w_3}\dfrac{\partial w_3}{\partial w_2}\right) \dfrac{\partial w_2}{\partial w_1} \right) \dfrac{\partial w_1}{\partial x} 
= \ldots
$$

* Obliczamy ***adjoint***: 

$$ \bar{w} = \frac{\partial f}{\partial w} $$

<img style="margin: auto" height="80%" src="ad2.png" />

## 2-layer Neural Network in Marian

```cpp
    auto x = input(shape={whatevs, 784});
    auto y = input(shape={whatevs, 10});
  
    auto w1 = param(shape={784, 100});
    auto b1 = param(shape={1, 100});
    auto l1 = tanh(dot(x, w1) + b1);
    
    auto w2 = param(shape={100, 10});
    auto b2 = param(shape={1, 10});
    auto l2 = softmax(dot(l1, w2) + b2, axis=1);
    
    auto graph = -mean(sum(y * log(l2), axis=1), 
                           axis=0);
  
    x = Tensor({500, 784}, 1);
    y = Tensor({500, 10}, 1);
  
    graph.forward();
    graph.backward();
  
    auto dw = w.grad();
    auto db = b.grad();
```

## Unary node for Tanh operation in Marian

```cpp
struct TanhNodeOp : public UnaryNodeOp {
  template <typename ...Args>
  TanhNodeOp(Args ...args)
  : UnaryNodeOp(args...) { }
  
  void forward() {
    Element(_1 = Tanh(_2),
            val_, a_->val());
  }
  
  void backward() {
    Element(_1 += _2 * (1 - Tanh(_3) * Tanh(_3)),
            a_->grad(), adj_, a_->val());
  }
};
```

### Źródła

* http://cs231n.github.io/optimization-2/
* https://github.com/emjotde/mt-marathon/blob/master/lec02/
* http://neuralnetworksanddeeplearning.com/chap2.html
* https://github.com/emjotde/Marian